In [1]:
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv('../.env'))

True

In [2]:
from langchain_core.documents.base import Document

In [3]:
# from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.vectorstores.pgvector import PGVector

# # use Azure OpenAI
# from langchain_openai import AzureOpenAIEmbeddings

In [4]:
from langchain.indexes import SQLRecordManager, index

In [5]:
import pandas as pd

In [6]:
# Thay thế {spreadsheet_id} và {gid} bằng giá trị thực của bạn
spreadsheet_id = '1heJgK2PGjMpCejkOEOGEFNNo8h7bRujWyOopTES9JR8'
# gid = '0' # AUB
gid = '1397755672' # US

# Tạo URL đúng
url = f'https://docs.google.com/spreadsheets/d/{spreadsheet_id}/export?format=csv&gid={gid}'

# Đọc dữ liệu từ Google Sheet
df = pd.read_csv(url)


In [7]:
df.head()

,question,answer
0,"In the students’ timetable, I have seen that t...","Besides lectures and tutorial sessions, studen..."
1,How can students improve their English if stud...,Improving students’ English skills is a top pr...


In [8]:
df.fillna("", inplace=True)
df.head()

,question,answer
0,"In the students’ timetable, I have seen that t...","Besides lectures and tutorial sessions, studen..."
1,How can students improve their English if stud...,Improving students’ English skills is a top pr...


In [9]:
data = [
    Document(
        page_content=row['question'], 
        metadata={
            'answer': row['answer'],
            # 'url': row['url_zalo'],
            # 'url_bus_image': row['url_bus_image'], 
            # 'url_link': row['url_link'], 
            # 'url_link2': row['url_link2']
        }
    )
    for index, row in df.iterrows()
]

In [10]:
data[0]

Document(metadata={'answer': 'Besides lectures and tutorial sessions, students will be required to engage in at least 10 hours of self-study per subject each week. This will include a considerable amount of time reading required textbooks, studying, and evaluating hand-outs, working on homework assignments, preparing for class work, revising for tests, and working in groups with fellow students on various projects. This requires students to learn to keep a strict time management plan which is essential to their engagement and success on the programme.'}, page_content='In the students’ timetable, I have seen that the students study about 2-4 hours a day only. Should this be enough to ensure quality and what should students do besides class time?')

#### connect to pgvector db and upload doc

In [11]:
host = os.getenv("DEMO_PG_VECTOR_HOST")
user = os.getenv("DEMO_PG_VECTOR_USER")
password = os.getenv("DEMO_PG_VECTOR_PASSWORD")

In [12]:
# database = os.getenv("DEMO_DATABASE_AUB_FREQUENTLYASKEDQUESTION") # AUB 
database = os.getenv("DEMO_DATABASE_US_FREQUENTLYASKEDQUESTION") # US

In [13]:
COLLECTION_NAME = "buv_demo_collection"

In [14]:
CONNECTION_STRING = f"postgresql+psycopg2://{user}:{password}@{host}:5432/{database}"
# CONNECTION_STRING

In [15]:
import sys
project_path = os.path.abspath(os.path.join('..'))
sys.path.append(project_path)

In [16]:
from models.chat import azure_openai
from models.embeddings import text_embedding_3large

In [17]:
vector_store = PGVector(
    embedding_function=text_embedding_3large,
    # embedding_function=embeddings,
    collection_name=COLLECTION_NAME,
    connection_string=CONNECTION_STRING,
)

C:\Users\vnt17\AppData\Local\Temp\ipykernel_31820\1767837163.py:1: LangChainPendingDeprecationWarning: This class is pending deprecation and may be removed in a future version. You can swap to using the `PGVector` implementation in `langchain_postgres`. Please read the guidelines in the doc-string of this class to follow prior to migrating as there are some differences between the implementations. See <https://github.com/langchain-ai/langchain-postgres> for details aboutthe new implementation.
  vector_store = PGVector(
C:\Users\vnt17\AppData\Local\Temp\ipykernel_31820\1767837163.py:1: LangChainPendingDeprecationWarning: Please use JSONB instead of JSON for metadata. This change will allow for more efficient querying that involves filtering based on metadata.Please note that filtering operators have been changed when using JSOB metadata to be prefixed with a $ sign to avoid name collisions with columns. If you're using an existing database, you will need to create adb migration for you

##### upload and update db

In [30]:
namespace = f"pgvector/{COLLECTION_NAME}"
record_manager = SQLRecordManager(
    namespace, db_url=CONNECTION_STRING
)

In [31]:
record_manager.create_schema()

In [32]:
index(
    data,
    record_manager,
    vector_store,
    cleanup="full",
    source_id_key="answer",
)

{'num_added': 2, 'num_updated': 0, 'num_skipped': 0, 'num_deleted': 0}

##### retriever

In [18]:
retriever = vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={
        "k": 1,
        "score_threshold": 0.5
    }          
)

In [19]:
# question = "how can i get timetable?" # aub
question = "How can students improve their English if students/parents would like to enhance their English skills?"

In [20]:
retrieved = retriever.get_relevant_documents(question)
retrieved

C:\Users\vnt17\AppData\Local\Temp\ipykernel_31820\3022424715.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use invoke instead.
  retrieved = retriever.get_relevant_documents(question)


[Document(metadata={'answer': 'Improving students’ English skills is a top priority at BUV, as our data shows that students with higher English proficiency tend to achieve better academic results. Our dedicated English Department offers a variety of support services to help students enhance their language abilities.'}, page_content='How can students improve their English if students/parents would like to enhance their English skills?\xa0')]

In [21]:
# retrieved[0].page_content
retrieved[0].metadata["answer"]

'Improving students’ English skills is a top priority at BUV, as our data shows that students with higher English proficiency tend to achieve better academic results. Our dedicated English Department offers a variety of support services to help students enhance their language abilities.'

### memory conversation

In [22]:
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

In [23]:
from langchain_core.prompts.base import BasePromptTemplate

In [24]:
from langchain.prompts import PromptTemplate

In [40]:
# prompt_template = BasePromptTemplate(
prompt_template = PromptTemplate(
    input_variables=['history', 'input'], 
    template="""The following is a friendly conversation between a human and an AI. \
    The AI is talkative and provides lots of specific details from its context. \
    If the AI does not know the answer to a question, it truthfully says it does not know.\n\n\
    always add 'yo dude' at the end of your answer\n\n
    Current conversation:\n{history}\nHuman: {input}\nAI:"""
)

In [41]:
conversation_buf = ConversationChain(
    llm=azure_openai,
    memory=ConversationBufferMemory(),
    prompt=prompt_template
)

C:\Users\vnt17\AppData\Local\Temp\ipykernel_15472\2329443293.py:1: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html instead.
  conversation_buf = ConversationChain(


In [42]:
conversation_buf("Good morning! My name is Ngoc")

C:\Users\vnt17\AppData\Local\Temp\ipykernel_15472\67019879.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  conversation_buf("Good morning! My name is Ngoc")


{'input': 'Good morning! My name is Ngoc',
 'history': '',
 'response': 'Good morning, Ngoc! How can I assist you today? Yo dude.'}

##### another test

In [25]:
retrieved = retriever.get_relevant_documents(question)

In [26]:
combine_doc = ""
for i in retrieved:
    combine_doc = combine_doc + str(i.metadata["answer"]) + "\n"
combine_doc

'Improving students’ English skills is a top priority at BUV, as our data shows that students with higher English proficiency tend to achieve better academic results. Our dedicated English Department offers a variety of support services to help students enhance their language abilities.\n'

In [32]:
template = f"""You are a chatbot specialized in answering frequently asked questions from customers about the student shuttle bus service \
provided by BUV.\n

Use exactly this following information as your answer: {combine_doc}

Current conversation:\n{{history}}\n
Human: {{input}}\nAI:"""


In [33]:
print(template)

You are a chatbot specialized in answering frequently asked questions from customers about the student shuttle bus service provided by BUV.


Use exactly this following information as your answer: Improving students’ English skills is a top priority at BUV, as our data shows that students with higher English proficiency tend to achieve better academic results. Our dedicated English Department offers a variety of support services to help students enhance their language abilities.


Current conversation:
{history}

Human: {input}
AI:


In [34]:
# prompt_template = BasePromptTemplate(
prompt_template = PromptTemplate(
    input_variables=['history', 'input'], 
    template=template
)

In [35]:
conversation_buf = ConversationChain(
    llm=azure_openai,
    memory=ConversationBufferMemory(),
    prompt=prompt_template,
    # input_types={
    #     "combine_doc": combine_doc
    # }
)

In [36]:
conversation_buf(question)

{'input': 'How can students improve their English if students/parents would like to enhance their English skills?',
 'history': '',
 'response': 'Improving students’ English skills is a top priority at BUV. Our dedicated English Department offers a variety of support services to help students enhance their language abilities. These services include English language courses, workshops, and tutoring sessions. Additionally, we provide access to online resources and language learning software to supplement classroom learning. Students and parents can also participate in English language immersion programs and cultural exchange activities to further improve their language skills.'}

In [37]:
# conversation_buf("how to receive it?")

{'input': 'how to receive it?',
 'history': 'Human: How can students improve their English if students/parents would like to enhance their English skills?\nAI: Improving students’ English skills is a top priority at BUV. Our dedicated English Department offers a variety of support services to help students enhance their language abilities. These services include English language courses, workshops, and tutoring sessions. Additionally, we provide access to online resources and language learning software to supplement classroom learning. Students and parents can also participate in English language immersion programs and cultural exchange activities to further improve their language skills.',
 'response': "To receive the support services offered by our English Department, students and parents can reach out to the department directly. They can inquire about English language courses, workshops, and tutoring sessions, and express their interest in participating. The department will provide 

In [ ]:
# conversation_buf.invoke({
#     "input": question
# })

In [40]:
conversation_buf.memory.buffer

"Human: How can students improve their English if students/parents would like to enhance their English skills?\nAI: Improving students’ English skills is a top priority at BUV. Our dedicated English Department offers a variety of support services to help students enhance their language abilities. These services include English language courses, workshops, and tutoring sessions. Additionally, we provide access to online resources and language learning software to supplement classroom learning. Students and parents can also participate in English language immersion programs and cultural exchange activities to further improve their language skills.\nHuman: how to receive it?\nAI: To receive the support services offered by our English Department, students and parents can reach out to the department directly. They can inquire about English language courses, workshops, and tutoring sessions, and express their interest in participating. The department will provide information on the available

In [41]:
conversation_buf.memory.clear()

In [42]:
conversation_buf.memory.buffer

''

In [43]:
question

'How can students improve their English if students/parents would like to enhance their English skills?'

In [44]:
conversation_buf.invoke({
    "input": question
})

{'input': 'How can students improve their English if students/parents would like to enhance their English skills?',
 'history': '',
 'response': 'Improving students’ English skills is a top priority at BUV. Our dedicated English Department offers a variety of support services to help students enhance their language abilities. These services include English language courses, workshops, and tutoring sessions. Additionally, we provide access to online resources and language learning software to supplement classroom learning. Students and parents can also participate in English language immersion programs and cultural exchange activities to further improve their language skills.'}

In [45]:
conversation_buf.memory.buffer

'Human: How can students improve their English if students/parents would like to enhance their English skills?\nAI: Improving students’ English skills is a top priority at BUV. Our dedicated English Department offers a variety of support services to help students enhance their language abilities. These services include English language courses, workshops, and tutoring sessions. Additionally, we provide access to online resources and language learning software to supplement classroom learning. Students and parents can also participate in English language immersion programs and cultural exchange activities to further improve their language skills.'